
# Bu soruya cevap arıyoruz: YOLO’ya (norm/attention/stride vb.) kavramlar nasıl entegre edilir?

Bu notebook şu 3 soruya teknik cevap verir:

1) **Norm / Attention / Stride / Downsample** gibi kavramlar YOLO’da *nereye* ve *nasıl* takılır?
2) Bu iş **deneme-yanılma mı**, yoksa gerçekten “kural” var mı?
3) Elimizdeki görüntü işleme verisine göre doğru adımlara **nasıl sezgi geliştiririz?**

> YOLO (v5/v8/v11) fark etmez: backbone–neck–head ayrımı değişse de tasarım mantığı aynıdır.



---

## 0) YOLO entegrasyon haritası: Backbone / Neck / Head

YOLO’yu 3 parçaya ayır:

1) **Backbone**: Feature çıkarır (C3/C4/C5)
2) **Neck**: Multi-scale fusion (FPN/PAN/PAFPN) → P3/P4/P5
3) **Head**: Tahmin (bbox + cls + obj)

Modül eklerken önce şunu belirle:
- Bu modül **feature extraction** mı iyileştiriyor? (Backbone)
- **feature fusion** mı? (Neck)
- **prediction** mı? (Head)

Genel pratik:
- **Norm**: backbone+neck boyunca
- **Attention**: backbone (orta/derin) ve neck’te seçici
- **Stride/Downsample**: backbone stage girişlerinde kritik



---

## 1) Norm (BN/GN/LN) entegrasyonu: kural gibi çalışan prensipler

### 1.1) Kural-1: Küçük batch + BatchNorm = risk
- BN istatistikleri batch’e bağlıdır.
- Küçük batch’te μ/σ gürültülü olur → eğitim dalgalanır.

### 1.2) Kural-2: Alternatifler
- **Multi-GPU**: SyncBN ile “effective batch” büyür
- **Tek GPU / küçük batch**: GroupNorm (batch bağımsız)

### 1.3) Kural-3: Değişim etkisi
Norm’u backbone/neck’te değiştirince feature dağılımı değişir.  
Head’de norm değişimi daha hassas olabilir; önce backbone/neck’te kazan.

Aşağıda hızlı bir seçim heuristiği var.


In [1]:

def choose_norm(batch_total, multi_gpu=False):
    if batch_total >= 16:
        return "BatchNorm (default)"
    if multi_gpu:
        return "SyncBatchNorm"
    return "GroupNorm"

for b in [2,4,8,16,32]:
    print(b, "->", choose_norm(b, multi_gpu=False))
print("8 (multi-gpu) ->", choose_norm(8, multi_gpu=True))


2 -> GroupNorm
4 -> GroupNorm
8 -> GroupNorm
16 -> BatchNorm (default)
32 -> BatchNorm (default)
8 (multi-gpu) -> SyncBatchNorm



---

## 2) Attention entegrasyonu: iki aileyi ayır

YOLO’da pratikte:

### A) Gating / Reweighting attention (SE/ECA/CBAM/CoordAtt)
- Pool → ağırlık üret → sigmoid → çarp
- CNN backbone/neck için “drop-in” ve görece hafif

### B) QKV / Self-attention
- \(A = softmax(QK^T/\sqrt{d})\), çıktı \(AV\)
- Daha ağır; backbone içine sınırlı sayıda koymak mantıklı

### Yerleştirme prensipleri
- **Erken katmanlarda** attention pahalıdır (H×W büyük).
- **Orta/derin katmanlar** daha verimli (H×W küçük).
- Neck, fusion yaptığı için attention için iyi adaydır.
- Head’de attention çoğu zaman son tercih.

Kısa kartlar:
- SE: kanal seçicilik
- CBAM: kanal + uzamsal
- CoordAtt: konumu daha iyi korur


In [2]:

gating_cards = [
    ("SE", "GAP → MLP → sigmoid → x*wc (channel)"),
    ("ECA", "GAP → 1D conv(ch) → sigmoid → x*wc"),
    ("CBAM", "Channel → Spatial (sigmoid mask)"),
    ("CoordAtt", "H ve W eksenlerinde ayrı özet → konum koru"),
]
for n, t in gating_cards:
    print(f"{n}: {t}")


SE: GAP → MLP → sigmoid → x*wc (channel)
ECA: GAP → 1D conv(ch) → sigmoid → x*wc
CBAM: Channel → Spatial (sigmoid mask)
CoordAtt: H ve W eksenlerinde ayrı özet → konum koru



---

## 3) Stride / Downsample: YOLO’da “en kritik düğme”

Tipik YOLO piramidi:
- P3: stride 8 (küçük nesneler)
- P4: stride 16
- P5: stride 32 (büyük nesneler)

### 3.1) Kural-1: Erken downsample küçük nesneyi öldürür
Dataset küçük nesne ağırlıklıysa:
- early stride agresif olmasın
- P3 kalitesi (feature) kritik

### 3.2) Kural-2: Downsample’u azaltınca compute patlar
YOLO gerçek zaman hedefli olduğu için latency/VRAM sınırın var.

Sonuç: stride planı bir **Pareto** problemidir (accuracy ↔ speed).

Aşağıdaki yardımcı, nesne piksel boyutuna göre hangi stride seviyesinin kritik olduğunu kaba önerir.


In [3]:

def stride_recommend(obj_px, strides=(8,16,32), min_cells=2):
    rec = []
    for s in strides:
        rec.append((s, obj_px/s))
    feasible = [r for r in rec if r[1] >= min_cells]
    if feasible:
        best = max(feasible, key=lambda x: x[0])
        return rec, f"Öneri: stride {best[0]} seviyesi yeterli (≈{best[1]:.2f} cell)."
    return rec, "Uyarı: Nesne çok küçük; stride 8 bile yetmiyor. Çözünürlük/augment/neck iyileştirme düşün."

for obj in [6,12,20,40,80]:
    rec, msg = stride_recommend(obj)
    print(f"obj≈{obj}px -> {rec} | {msg}")


obj≈6px -> [(8, 0.75), (16, 0.375), (32, 0.1875)] | Uyarı: Nesne çok küçük; stride 8 bile yetmiyor. Çözünürlük/augment/neck iyileştirme düşün.
obj≈12px -> [(8, 1.5), (16, 0.75), (32, 0.375)] | Uyarı: Nesne çok küçük; stride 8 bile yetmiyor. Çözünürlük/augment/neck iyileştirme düşün.
obj≈20px -> [(8, 2.5), (16, 1.25), (32, 0.625)] | Öneri: stride 8 seviyesi yeterli (≈2.50 cell).
obj≈40px -> [(8, 5.0), (16, 2.5), (32, 1.25)] | Öneri: stride 16 seviyesi yeterli (≈2.50 cell).
obj≈80px -> [(8, 10.0), (16, 5.0), (32, 2.5)] | Öneri: stride 32 seviyesi yeterli (≈2.50 cell).



---

## 4) Deneme-yanılma mı, kural mı?

**Cevap:** İkisi beraber.

### 4.1) “Kural” tarafı = hard constraints
1) Latency/VRAM bütçesi  
2) Multi-scale piramit (P3/P4/P5) gereksinimi  
3) Batch–Norm uyumu (küçük batch + BN)  
4) Küçük nesne varsa erken stride agresif olamaz  

### 4.2) “Deney” tarafı = controlled ablation
Deneme-yanılma kötü değil; **kontrolsüz** olunca kötü.

Kural:
- Tek seferde **tek değişken**
- Aynı seed/schedule/augment
- Aynı metrikler (mAP50-95, mAP_s/m/l, FPS/latency, VRAM)

Aşağıda örnek ablation planı.


In [4]:

try:
    import pandas as pd
    plan = [
        {"Exp": "E0", "Değişiklik": "Baseline", "Beklenti": "Referans", "Ölç": "mAP_s/m/l, FPS, VRAM"},
        {"Exp": "E1", "Değişiklik": "BN→GN", "Beklenti": "Küçük batch stabil", "Ölç": "mAP + run-to-run std"},
        {"Exp": "E2", "Değişiklik": "Neck: CBAM", "Beklenti": "Fusion iyileşir", "Ölç": "mAP_small, FP analizi"},
        {"Exp": "E3", "Değişiklik": "Backbone: SE (mid)", "Beklenti": "Channel seçicilik", "Ölç": "mAP, latency"},
        {"Exp": "E4", "Değişiklik": "Downsample yöntemi", "Beklenti": "Aliasing azalır", "Ölç": "mAP_small + görsel hata"},
    ]
    display(pd.DataFrame(plan))
except Exception:
    pass


,Exp,Değişiklik,Beklenti,Ölç
0,E0,Baseline,Referans,"mAP_s/m/l, FPS, VRAM"
1,E1,BN→GN,Küçük batch stabil,mAP + run-to-run std
2,E2,Neck: CBAM,Fusion iyileşir,"mAP_small, FP analizi"
3,E3,Backbone: SE (mid),Channel seçicilik,"mAP, latency"
4,E4,Downsample yöntemi,Aliasing azalır,mAP_small + görsel hata



---

## 5) Sezgi nasıl gelişir? (veriden tasarıma)

Sezgi = **veri istatistiği + hata analizi + maliyet bilinci**

### 5.1) Veri istatistiği (ilk bakılacaklar)
- Nesne boyutu dağılımı (small/medium/large)
- Aspect ratio dağılımı
- Sınıf dengesizliği
- Arka plan karmaşıklığı (clutter)
- Blur/noise/düşük ışık

### 5.2) Hata analizi (ikinci adım)
- FN mi fazla (kaçırma) yoksa FP mi?
- Küçük nesne kaçırma baskın mı?
- Yakın nesneleri ayıramama (NMS / localization) mı?
- Sınıf karışması mı?

### 5.3) Tasarım eşleştirme (üçüncü adım)
- Küçük nesne FN → P3 kalite + neck fusion + (gerekirse) dikkatli stride değişimi
- FP/arka plan → spatial attention + augmentation + veri temizliği
- Localization hatası → box loss/IoU türleri + head ayarı + label kalitesi



---

## 6) Uygulama rehberi: en düşük riskten başla

YOLO’da modül eklerken çoğu zaman en iyi sıra:

1) **Baseline’ı sabitle** (split, augment, schedule, seed)
2) **Norm** (BN↔GN/SyncBN)  
3) **Neck iyileştirmeleri** (hafif attention)  
4) **Backbone mid-stage attention**  
5) **Stride/downsample** (en pahalı ve riskli değişim)

> Bu sıra rastgele değil: “en çok sistem etkileyen” (stride) en sona kalır.



---

## 7) Kapanış (tek paragraf)

YOLO’ya norm/attention/stride gibi kavramlar rastgele eklenmez.
Kurallar, mühendislik sınırlarıdır: multi-scale piramit, compute bütçesi, batch–norm uyumu ve küçük nesne/erken stride ilişkisi.
Geri kalan kısım “deneme-yanılma” değil, **kontrollü ablation** ile ölçülür.
Sezgi, veri ölçeklerini (nesne boyutu), hata tiplerini (FP/FN) ve latency/VRAM maliyetini birlikte okuyup
modülü doğru yere (backbone/neck/head) ve doğru şiddette koydukça oluşur.
